# Write ATS input files

In [1]:
# these can be turned on for development work
%load_ext autoreload
%autoreload 2

In [2]:
# setting up logging first or else it gets preempted by another package
import watershed_workflow.ui
watershed_workflow.ui.setup_logging(1)

In [3]:
import os,sys
import logging
import pickle
import numpy as np

import pandas as pd
import geopandas as gpd
import cftime, datetime
pd.options.display.max_columns = None

import watershed_workflow 
import watershed_workflow.config
import watershed_workflow.sources
import watershed_workflow.utils
import watershed_workflow.plot
import watershed_workflow.mesh
import watershed_workflow.regions
import watershed_workflow.meteorology
import watershed_workflow.land_cover_properties
import watershed_workflow.resampling
import watershed_workflow.condition
import watershed_workflow.io
import watershed_workflow.sources.standard_names as names

import ats_input_spec
import ats_input_spec.public
import ats_input_spec.io

import amanzi_xml.utils.io as aio
import amanzi_xml.utils.search as asearch
import amanzi_xml.utils.errors as aerrors




In [4]:
# Force Watershed Workflow to pull data from this directory rather than a shared data directory.
# This picks up the Coweeta-specific datasets set up here to avoid large file downloads for 
# demonstration purposes.
#
def splitPathFull(path):
    """
    Splits an absolute path into a list of components such that
    os.path.join(*splitPathFull(path)) == path
    """
    parts = []
    while True:
        head, tail = os.path.split(path)
        if head == path:  # root on Unix or drive letter with backslash on Windows (e.g., C:\)
            parts.insert(0, head)
            break
        elif tail == path:  # just a single file or directory
            parts.insert(0, tail)
            break
        else:
            parts.insert(0, tail)
            path = head
    return parts

cwd = splitPathFull(os.getcwd())
assert cwd[-1] == 'workflow'
cwd = cwd[:-1]

# Note, this directory is where downloaded data will be put as well
data_dir = os.path.join(*(cwd + ['input_data',]))
def toInput(filename):
    return os.path.join(data_dir, filename)

output_filenames = dict()
output_dir = os.path.join(*(cwd + ['output_data',]))
def fromOutput(filename):
    return os.path.join(output_dir, filename)    

def toOutput(role, filename):
    output_filenames[role] = filename
    return fromOutput(filename)

def pathReltoRun(filename):
    """Strips the absolute path and sets it relative to the run directory, for use in input xml"""
    fs = splitPathFull(filename)
    fs = fs[len(cwd):]
    return os.path.join(*(['..', '..',] + fs))

# check output and input dirs exist
if not os.path.isdir(data_dir):
    os.makedirs(data_dir, exist_ok=True)
if not os.path.isdir(output_dir):
    os.makedirs(output_dir, exist_ok=True)
       

In [5]:
# Set the data directory to the local space to get the locally downloaded files
# REMOVE THIS CELL for general use outside fo Coweeta
watershed_workflow.config.setDataDirectory(data_dir)


In [6]:
## Parameters cell -- this provides all parameters that can be changed via pipelining to generate a new watershed. 
name = 'RussianRiver'
hucs = ['18010110'] # a list of HUCs to run

# Geometric parameters
# -- parameters to clean and reduce the river network prior to meshing
prune_by_area = 10               # km^2
simplify = 125                   # length scale to target average edge 

# -- mesh triangle refinement control
refine_d0 = 200
refine_d1 = 600

refine_L0 = 125
refine_L1 = 300

refine_A0 = refine_L0**2 / 2
refine_A1 = refine_L1**2 / 2

# Refine triangles if they get too acute
min_angle = 20 # degrees

# width of reach by stream order (order:width)
river_widths = dict({1:10, 2:10, 3:20, 4:30, 5:30}) 


# Note that, by default, we tend to work in the DayMet CRS because this allows us to avoid
# reprojecting meteorological forcing datasets.
crs = watershed_workflow.crs.default_crs


# start and stop time for simulation
# note that this is the overlap of AORC and MODIS
start = cftime.DatetimeGregorian(2007, 8, 1)
end = cftime.DatetimeGregorian(2020, 7, 31)

start_noleap = cftime.DatetimeNoLeap(2007, 8, 1)
end_noleap = cftime.DatetimeNoLeap(2020, 7, 31)
cyclic_nyears = 10


# Global Soil Properties
min_porosity = 0.05 # minimum porosity considered "too small"
max_permeability = 1.e-10 # max value considered "too permeable"
max_vg_alpha = 1.e-3 # max value of van Genuchten's alpha -- our correlation is not valid for some soils

## Read data from previous runs

In [7]:
# read in filenames
filenames = dict()
for run in range(1, 7):
    local_filenames_name = fromOutput(f'{run:02}_output_filenames.txt')
    with open(local_filenames_name, 'rb') as fid:
        local_dict = pickle.load(fid)
        filenames.update(local_dict)

display(pd.DataFrame({'keys':filenames.keys(), 'filenames':filenames.values()}))



,keys,filenames
0,watershed_polys,02_watersheds.pickle
1,rivers,02_rivers.parquet
2,evaluation_discharge,01_discharge_observations.csv
3,01_output_filenames,01_output_filenames.txt
4,02_output_filenames,02_output_filenames.txt
5,m2,04_m2.pickle
6,03_output_filenames,03_output_filenames.txt
7,land_cover_properties,04_land_cover_properties.csv
8,modis_lai_typical,RussianRiver_LAI_MODIS_CyclicSteadystate_1997_...
9,modis_lai_ts,RussianRiver_LAI_MODIS_Transient_2007_2020.h5


In [8]:
print(fromOutput(filenames['nlcd_lai_typical']))
mesh_local_filename = pathReltoRun(fromOutput(filenames['mesh']))
print(mesh_local_filename)

/home/ecoon/research/HypeRFACETs/PacWest-RDT/RussianRiver-smaller/output_data/RussianRiver_LAI_NLCD_CyclicSteadystate_1997_2007.h5
../../output_data/RussianRiver.exo


## Helper functions for creating input files

In [9]:
# Note that each of these are defined as functions so we can reuse them for all three input files.

# add the subsurface and surface domains
#
# Note this also adds a "computational domain" region to the region list, and a vis spec 
# for "domain"
def add_domains(main_list, mesh, surface_region='surface', snow=True, canopy=True):
    ats_input_spec.public.add_domain(main_list, 
                                 domain_name='domain', 
                                 dimension=3, 
                                 mesh_type='read mesh file',
                                 mesh_args={'file':mesh})
    if surface_region:
        main_list['mesh']['domain']['build columns from set'] = surface_region    
    
        # Note this also adds a "surface domain" region to the region list and a vis spec for 
        # "surface"
        ats_input_spec.public.add_domain(main_list,
                                domain_name='surface',
                                dimension=2,
                                mesh_type='surface',
                                mesh_args={'surface sideset name':'surface'})
    if snow:
        # Add the snow and canopy domains, which are aliases to the surface
        ats_input_spec.public.add_domain(main_list,
                                domain_name='snow',
                                dimension=2,
                                mesh_type='aliased',
                                mesh_args={'target':'surface'})
    if canopy:
        ats_input_spec.public.add_domain(main_list,
                                domain_name='canopy',
                                dimension=2,
                                mesh_type='aliased',
                                mesh_args={'target':'surface'})

In [10]:

def round_dataframe(df, significant_digits):
    def round_sig(x, sig=significant_digits):
        if np.isnan(x):
            return x
        return round(x, sig - int(np.floor(np.log10(abs(x)))) - 1) if x != 0 else 0

    return df.applymap(lambda x: round_sig(x, 6) if isinstance(x, (float, np.floating)) else x)


In [11]:
def add_land_cover(main_list):
    lc = pd.read_csv(fromOutput('04_land_cover_properties.csv')).set_index('indices')

    # next write a land-cover section for each NLCD type
    for nlcd_index, nlcd_name in zip(lc.index, lc['names']):
        ats_input_spec.public.add_land_cover(main_list, nlcd_name, nlcd_index, mesh_local_filename, **lc.loc[nlcd_index])
   
    # override Mannings n by stream order
    fe = main_list['state']['evaluators']['surface-manning_coefficient']
    n_by_order = watershed_workflow.land_cover_properties.mannings_n_by_stream_order
    for i in range(10):
        if f'stream order {i}' in main_list['regions']:
            sublist = fe['function'].append_empty(f'stream order {i}')
            sublist['region'] = f'stream order {i}'
            sublist['component'] = 'cell'
            entry = sublist['function'].set_type('constant', ats_input_spec.public.known_specs['function-constant-spec'])
            entry['value'] = n_by_order.loc[min(i, 5), "Manning's n [-]"]



In [12]:
# add soil sets: note we need a way to name the set, so we use, e.g. SSURGO-MUKEY.
def add_soil_properties(main_list):
    # add soil material ID regions, porosity, permeability, and WRMs
    subsurface_props_used = pd.read_csv(fromOutput(filenames['subsurface properties']))

    subsurface_props_used = round_dataframe(subsurface_props_used, 6)    
    subsurface_props_used = subsurface_props_used.set_index('ats_id')
    
    for ats_id in subsurface_props_used.index:
        props = subsurface_props_used.loc[ats_id]
        set_name = props['name']
        
        if props['van Genuchten n [-]'] < 1.5:
            smoothing_interval = 0.01
        else:
            smoothing_interval = 0.0
        
        ats_input_spec.public.add_soil_type(main_list, set_name, ats_id, mesh_local_filename,
                                            float(props['porosity [-]']),
                                            float(props['permeability [m^2]']), 1.e-7,
                                            float(props['van Genuchten alpha [Pa^-1]']),
                                            float(props['van Genuchten n [-]']),
                                            float(props['residual saturation [-]']),
                                            float(smoothing_interval))    


In [13]:
def parseLabeledSets(main_list):
    # add labeled sets
    with open(fromOutput(filenames['labeled set metadata']), 'rb') as fid:
        lss = pickle.load(fid)

    for ls in lss:
        ls_name, ls_setid, ls_entity = ls

        if ls_setid < 10 or ls_setid >= 10000:
            # fixes prior bug in this workflow -- outlet regions should be called "DOMAIN surface outlet", not "DOMAIN outlet"
            if ls_name.endswith("outlet") and ls_name != "surface domain outlet" and ls_name != "RR-outlet":
                ls_name = ls_name[0:-len("outlet")]+"surface outlet"
            
            ats_input_spec.public.add_region_labeled_set(main_list, ls_name, ls_setid, mesh_local_filename, ls_entity)     



In [14]:
with open(fromOutput('02_watersheds.pickle'), 'rb') as fid:
    watersheds = pickle.load(fid)

watersheds.df


,geometry,station_nm,dec_lat_va,dec_long_va,alt_va,alt_acy_va,alt_datum_cd,huc_cd,begin_date,end_date,units,tz,ID,count,comid,measure,name,reach_ID,river_index,reach_index,location_on_reach,true_geometry,incremental_catchment,true_gage_geometry,outlet
23,"POLYGON ((-2313554.994 2045820.82, -2313505.56...",Russian River Outlet,NaN,NaN,NaN,NaN,None,None,None,None,None,None,RR-outlet,NaN,8272933,0.0000,RR-outlet,8272933,0,0,0,POINT (-2313594.047 2045820.756),"POLYGON ((-2305777.792 2037720.003, -2305971.4...",POINT (-2313594.047 2045820.756),POINT (-2313594.047 2045820.756)
21,"POLYGON ((-2298855.021 2036715.003, -2298543.4...","Russian R A Hacienda Bridge Nr Guerneville, CA",38.508482,-122.927734,22.33,0.16,NAVD88,18010110,1939-10-01,2025-11-25,mm/day,UTC,USGS-11467000,9461.0,8272725,84.0061,USGS-11467000,8272725,0,40,1,POINT (-2295059.53 2047164.785),"POLYGON ((-2296796.653 2031061.892, -2296895.0...",POINT (-2295059.53 2047164.785),POINT (-2294250.425 2047087.173)
10,"POLYGON ((-2285054.948 2060745.035, -2284871.6...","Russian R Nr Healdsburg, CA",38.613203,-122.836345,79.98,0.26,NAVD88,18010110,1939-10-01,2025-11-25,mm/day,UTC,USGS-11464000,9461.0,8272379,89.9621,USGS-11464000,8272379,0,135,1,POINT (-2284208.116 2056234.303),"POLYGON ((-2282295.004 2053875.042, -2282306.1...",POINT (-2284208.116 2056234.303),POINT (-2283791.112 2056134.658)
8,"POLYGON ((-2289285.013 2070075.002, -2289246.7...","Russian R A Jimtown, CA",38.658202,-122.829401,143.70,0.26,NAVD88,18010110,2009-07-01,2025-11-25,mm/day,UTC,USGS-11463682,5327.0,8272211,90.8312,USGS-11463682,8272211,0,152,1,POINT (-2282227.758 2060911.556),"POLYGON ((-2282169.29 2061204.75, -2282340.484...",POINT (-2282227.758 2060911.556),POINT (-2282342.202 2061057.323)
7,"POLYGON ((-2298704.965 2090475.004, -2298957.0...","Russian R A Geyserville, CA",38.712645,-122.895792,165.68,0.26,NAVD88,18010110,1910-10-01,2025-11-25,mm/day,UTC,USGS-11463500,4596.0,8272107,28.1658,USGS-11463500,8272107,0,158,1,POINT (-2286019.829 2068367.397),"POLYGON ((-2287362.021 2074784.464, -2287412.6...",POINT (-2286019.829 2068367.397),POINT (-2287482.499 2070478.45)
4,"POLYGON ((-2306505.014 2113815.062, -2306474.7...","Russian R Nr Cloverdale, CA",38.879308,-123.053577,374.64,0.16,NAVD88,18010110,1951-08-01,2025-11-25,mm/day,UTC,USGS-11463000,9459.0,8271445,82.7127,USGS-11463000,8271445,0,187,1,POINT (-2293821.342 2090102.265),"POLYGON ((-2299493.378 2094164.414, -2299640.0...",POINT (-2293821.342 2090102.265),POINT (-2293880.874 2090543.664)
3,"POLYGON ((-2306925.004 2123204.975, -2306447.9...","Russian R Nr Hopland, CA",39.026527,-123.130525,500.01,0.26,NAVD88,18010110,1939-10-01,2025-11-25,mm/day,UTC,USGS-11462500,9461.0,8269119,91.8680,USGS-11462500,8269119,0,218,1,POINT (-2295520.729 2107795.032),"POLYGON ((-2299591.085 2107888.87, -2299735.69...",POINT (-2295520.729 2107795.032),POINT (-2295539.859 2108307.778)
2,"POLYGON ((-2306925.004 2123204.975, -2306447.9...","Russian R Nr Talmage, CA",39.112913,-123.183028,545.95,0.16,NAVD88,18010110,2009-08-06,2025-11-25,mm/day,UTC,USGS-11462080,5956.0,8268521,11.7142,USGS-11462080,8268521,0,235,0,POINT (-2297109.406 2118353.355),"POLYGON ((-2298014.974 2118944.952, -2298210.8...",POINT (-2297109.406 2118353.355),POINT (-2297141.368 2118153.588)
14,"POLYGON ((-2298734.95 2054685.032, -2298344.05...","Dry C Nr Mouth Nr Healdsburg, CA",38.587370,-122.862178,56.70,0.16,NAVD88,18010110,1981-10-01,2025-11-25,mm/day,UTC,USGS-11465350,7790.0,8272363,2.3004,USGS-11465350,8272363,0,55,0,POINT (-2287155.73 2054074.887),"POLYGON ((-2287719.891 2054198.95, -2287880.08...",POINT (-2287155.73 2054074.887),POINT (-2287131.618 2054099.621)
13,"POLYGON ((-2294444.975 2063565.002, -2294725.1...","Dry C Blw Lambert Br Nr Geyserville, CA",38.653202,-122.927181,112.03,0.16,NAVD88,18010110,2011-11-10,2025-11-25,mm/day,UTC,USGS-11465240,4778.0,8272227,68.2805,USGS-11465240,8272227,0,66,1,POINT (-2290478.701 2062725.744),"POLYGON ((-2292170.937 2062428.575, -2292194.9...",POINT

In [15]:
# get an ATS "main" input spec list -- note, this is a dummy and is not used to write any files yet
def get_main(steadystate=False):
    main_list = ats_input_spec.public.get_main()

    # add the mesh and all domains
    if steadystate:
        add_domains(main_list, mesh_local_filename, canopy=False, snow=False)
    else:
        add_domains(main_list, mesh_local_filename)
        
    # add labeled sets
    parseLabeledSets(main_list)
         
    # add land cover for each nlcd
   
    add_land_cover(main_list)

    # add soil properties
    add_soil_properties(main_list)
        
    # add observations for each subcatchment
    ats_input_spec.public.add_observations_water_balance(main_list, "computational domain", 
                                                        "surface domain", "external sides",
                                                        steadystate=steadystate)

    with open(fromOutput('02_watersheds.pickle'), 'rb') as fid:
        watersheds = pickle.load(fid)    

    for ws in watersheds.df['ID']:
        ats_input_spec.public.add_observations_water_balance(main_list, ws, steadystate=steadystate)
    
    return main_list

In [16]:
def populate_basic_properties(xml, main_xml):
    """This function updates an xml object with the above properties for mesh, regions, soil props, and lc props"""
    # find and replace the mesh list
    xml.replace('mesh', asearch.child_by_name(main_xml, 'mesh'))

    # find and replace the regions list
    xml.replace('regions', asearch.child_by_name(main_xml, 'regions'))

    # update the observations list
    obs = next(i for (i,el) in enumerate(xml) if el.get('name') == 'observations')
    xml[obs] = asearch.child_by_name(main_xml, 'observations')

    # update all model parameters lists
    xml_parlist = asearch.find_path(xml, ['state', 'model parameters'], no_skip=True)
    for parlist in asearch.find_path(main_xml, ['state', 'model parameters'], no_skip=True):
        try:
            xml_parlist.replace(parlist.getName(), parlist)
        except aerrors.MissingXMLError:
            xml_parlist.append(parlist)

    # update all evaluator lists
    xml_elist = asearch.find_path(xml, ['state', 'evaluators'], no_skip=True)
    for elist in asearch.find_path(main_xml, ['state', 'evaluators'], no_skip=True):
        try:
            xml_elist.replace(elist.getName(), elist)
        except aerrors.MissingXMLError:
            xml_elist.append(elist)    
    
    # find and replace land cover
    mp_list = asearch.find_path(xml, ['state', 'model parameters'], no_skip=True)
    lc_list = asearch.find_path(main_xml, ['state', 'model parameters', 'land cover types'], no_skip=True)
    
    try:
        mp_list.replace('land cover types', lc_list)
    except aerrors.MissingXMLError:
        mp_list.append(lc_list)


## Write the files

### Steadystate spinup step

In [17]:
# create the main list
main = get_main(steadystate=True)

# set precip to 0.6 * the mean precip value
precip = main['state']['evaluators'].append_empty('surface-precipitation')
with open(fromOutput(filenames['total_avg_precip']), 'rb') as fid:
    precip_mean = pickle.load(fid)

precip.set_type('independent variable constant', ats_input_spec.public.known_specs['evaluator-independent-variable-constant-spec'])
precip['value'] = float(precip_mean * .6)
    
# load the template file
prefix = 'steadystate'
xml = aio.fromFile(toInput(f'{prefix}-template.xml'))
    
# update the template xml with the main xml generated here
main_xml = ats_input_spec.io.to_xml(main)
populate_basic_properties(xml, main_xml)


# create a run directory
os.makedirs(os.path.join('..', f'00_{name}_{prefix}', 'run0'), exist_ok=True)

#print(xml)

# write to disk
aio.toFile(xml, os.path.join('..', f'00_{name}_{prefix}', f'00_{name}_{prefix}.xml'))


            

/home/ecoon/code/ats/ats_input_spec/repos/master/ats_input_spec/io.py:43: UserWarning: Creating an incomplete XML object, missing entries!
  warnings.warn('Creating an incomplete XML object, missing entries!')


In [18]:
print(main)


cycle driver [coordinator-spec] : [incomplete]
  ONE OF:
    end time [double] : None
    end time units [string] : "s"
  OR:
    end cycle [int] : [optional]
  start time [double] : 0
  start time units [string] : "s"
  subcycled timestep [bool] : false
  restart from checkpoint file [string] : [optional]
  wallclock duration [hrs] [double] : [optional]
  required times [io-event-spec] : [optional]
    cycles start period stop [Array(int)] : [optional]
    cycles start period stop 0 [Array(int)] : [optional]
    cycles [Array(int)] : [optional]
    times start period stop [Array(double)] : [optional]
    times start period stop units [string] : s
    times start period stop 0 [Array(double)] : [optional]
    times start period stop 0 units [string] : s
    times [Array(double)] : [optional]
    times units [string] : s
  PK tree [PK-typed-spec-list] : [incomplete]
    
mesh [mesh-typed-spec-list] : [complete]
  domain [TypedCollection] : [complete]
    mesh type [string] : read mesh f

### Cyclic steadystate spinup step

In [19]:
subsurface_props_used = pd.read_csv(fromOutput(filenames['subsurface properties']))
subsurface_props_used

,Unnamed: 0,ats_id,ID,name,source,permeability [m^2],porosity [-],van Genuchten alpha [Pa^-1],van Genuchten n [-],residual saturation [-],geometry,mukey,thickness [m]
0,0,100,"(1791548,)",GLHYMPS-1791548,GLHYMPS,6.309573e-16,0.190000,0.000025,2.000000,0.010000,MULTIPOLYGON (((-2282205.9667936414 2136713.31...,NaN,NaN
1,1,101,"(656259,)",GLHYMPS-656259,GLHYMPS,6.309573e-16,0.190000,0.000025,2.000000,0.010000,MULTIPOLYGON (((-2304541.4293391607 2146574.82...,NaN,NaN
2,2,103,"(657155,)",GLHYMPS-657155,GLHYMPS,6.309573e-16,0.190000,0.000025,2.000000,0.010000,POLYGON ((-2281805.109329821 2139493.509268224...,NaN,NaN
3,3,104,"(657225,)",GLHYMPS-657225,GLHYMPS,1.000000e-13,0.220000,0.000294,2.000000,0.010000,MULTIPOLYGON (((-2286477.799981128 2138922.959...,NaN,NaN
4,4,105,"(657247,)",GLHYMPS-657247,GLHYMPS,1.000000e-13,0.220000,0.000294,2.000000,0.010000,MULTIPOLYGON (((-2296081.46975232 2137750.0572...,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
536,536,1398,3115319,NRCS-3115319,NRCS,8.530679e-13,0.340084,0.000100,1.400788,0.207362,MULTIPOLYGON (((-2317275.0261240094 2092758.57...,3115319.0,0.512577
537,537,1399,3115322,NRCS-3115322,NRCS,1.172957e-12,0.214882,0.000107,1.366197,0.222654,MULTIPOLYGON (((-2310357.7392605837 2134103.29...,3115322.0,1.371500
538,538,1400,3251971,NRCS-3251971,NRCS,1.636090e-13,0.439850,0.000085,1.308479,0.265837,MULTIPOLYGON (((-2285692.2989554736 2023919.95...,3251971.0,1.882600
539,539,1401,3251972,NRCS-3251972,NRCS,1.636090e-13,0.439850,0.000085,1.308479,0.265837,MULTIPOLYGON (((-2278859.508434973 2025515.845...,3251972.0,1.882600


### Transient simulation

In [20]:
subsurface_props_used = pd.read_csv(fromOutput(filenames['subsurface properties']))
display(subsurface_props_used)

,Unnamed: 0,ats_id,ID,name,source,permeability [m^2],porosity [-],van Genuchten alpha [Pa^-1],van Genuchten n [-],residual saturation [-],geometry,mukey,thickness [m]
0,0,100,"(1791548,)",GLHYMPS-1791548,GLHYMPS,6.309573e-16,0.190000,0.000025,2.000000,0.010000,MULTIPOLYGON (((-2282205.9667936414 2136713.31...,NaN,NaN
1,1,101,"(656259,)",GLHYMPS-656259,GLHYMPS,6.309573e-16,0.190000,0.000025,2.000000,0.010000,MULTIPOLYGON (((-2304541.4293391607 2146574.82...,NaN,NaN
2,2,103,"(657155,)",GLHYMPS-657155,GLHYMPS,6.309573e-16,0.190000,0.000025,2.000000,0.010000,POLYGON ((-2281805.109329821 2139493.509268224...,NaN,NaN
3,3,104,"(657225,)",GLHYMPS-657225,GLHYMPS,1.000000e-13,0.220000,0.000294,2.000000,0.010000,MULTIPOLYGON (((-2286477.799981128 2138922.959...,NaN,NaN
4,4,105,"(657247,)",GLHYMPS-657247,GLHYMPS,1.000000e-13,0.220000,0.000294,2.000000,0.010000,MULTIPOLYGON (((-2296081.46975232 2137750.0572...,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
536,536,1398,3115319,NRCS-3115319,NRCS,8.530679e-13,0.340084,0.000100,1.400788,0.207362,MULTIPOLYGON (((-2317275.0261240094 2092758.57...,3115319.0,0.512577
537,537,1399,3115322,NRCS-3115322,NRCS,1.172957e-12,0.214882,0.000107,1.366197,0.222654,MULTIPOLYGON (((-2310357.7392605837 2134103.29...,3115322.0,1.371500
538,538,1400,3251971,NRCS-3251971,NRCS,1.636090e-13,0.439850,0.000085,1.308479,0.265837,MULTIPOLYGON (((-2285692.2989554736 2023919.95...,3251971.0,1.882600
539,539,1401,3251972,NRCS-3251972,NRCS,1.636090e-13,0.439850,0.000085,1.308479,0.265837,MULTIPOLYGON (((-2278859.508434973 2025515.845...,3251972.0,1.882600
